In [11]:
# from aicsimageio import AICSImage
import numpy as np
import napari
from skimage.transform import resize
import glob2 as glob
import os
import pandas as pd
import skimage.io as io
import plotly.express as px
# read the image data


# set paths
# root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\pecfin_dynamics\\fin_morphodynamics\\"
root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/fin_morphodynamics/"
experiment_date = "20231214"

# make figure path
figure_path = os.path.join(root, "figures", experiment_date, "")
if not os.path.isdir(figure_path):
    os.makedirs(figure_path)
    
# load metadata file
metadata_path = os.path.join(root, "metadata", experiment_date + "_master_metadata_df.csv")
metadata_df = pd.read_csv(metadata_path, index_col=0)
metadata_df.head(10)

,nd2_series,well_id,genotype,start_age_hpf,time_index,well_index,time,stage_z_um,stage_y_um,stage_x_um,x_res_um,y_res_um,z_res_um,notes,example_flag,follow_up_flag,time_string,qc_flag,estimated_stage_hpf
0,1,A00,wik-inj-ctrl,48.0,0,0,0.662084,7024.96,-31737.6,48703.3,0.275,0.275,2.0,partial rotationevident,1,0,t000,1.0,48.000184
1,2,B00,wik-inj-ctrl,48.0,0,1,84.727853,7102.76,-22708.8,48919.4,0.275,0.275,2.0,fin tip moves beyond FOV pretty early on,1,0,t000,1.0,48.023536
2,3,C00,wik-inj-ctrl,48.0,0,2,164.422052,7076.20,-13654.0,48053.7,0.275,0.275,2.0,NaN,0,0,t000,1.0,48.045673
3,4,D00,wik-inj-ctrl,48.0,0,3,244.597807,7043.10,-4530.9,49390.5,0.275,0.275,2.0,NaN,0,0,t000,1.0,48.067944
4,5,E00,wik-inj-ctrl,48.0,0,4,325.039136,7039.76,4616.8,48572.3,0.275,0.275,2.0,NaN,0,0,t000,1.0,48.090289
5,6,F00,wik-inj-ctrl,48.0,0,5,406.321102,7037.20,13788.4,49308.6,0.275,0.275,2.0,NaN,0,0,t000,1.0,48.112867
6,7,G00,wik-inj-ctrl,48.0,0,6,484.849241,7004.24,22798.2,48747.8,0.275,0.275,2.0,NaN,0,0,t000,1.0,48.134680
7,8,H00,wik-inj-ctrl,48.0,0,7,565.973182,6949.58,31745.1,48300.9,0.275,0.275,2.0,NaN,0,0,t000,1.0,48.157215
8,9,H01,prdm1a,48.0,0,8,650.190232,6975.06,31770.5,40815.1,0.275,0.275,2.0,"Delayed. Also, strange curled morphology. Base...",1,0,t000,3.0,48.180608
9,10,G01,prdm1a,48.0,0,9,731.322439,6966.58,22676.9,39499.1,0.275,0.275,2.0,Lumpy leading eadge. Delayed. Possible it is n...,1,0,t000,4.0,48.203145


In [8]:
np.unique(metadata_df["qc_flag"])

array([-5. , -4. , -3. , -2.5, -2. , -1. ,  0. ,  1. ,  2. ,  2.5,  3. ,
        4. ,  5. ,  6. ,  7. ,  nan])

#### First, make a heatmap showing coverage for each genotype by developmental time

In [14]:
nn_indices = np.where(~np.isnan(metadata_df["qc_flag"]))[0]

fig = px.histogram(metadata_df.iloc[nn_indices], x="estimated_stage_hpf", color="genotype",
                  labels={"estimated_stage_hpf": "approximate stage (hpf)"}, template="plotly")

fig.update_layout(yaxis_title="number of embryos")
fig.show()
fig.write_image(figure_path + "data_coverage_summary.png", scale=2)

In [56]:
# generate coarse stage vec
stage_vec = metadata_df["estimated_stage_hpf"]
res = 2
stage_vec_rounded = np.round(stage_vec/2)*res
metadata_df["stage_rounded"] = stage_vec_rounded

# generate presence/absence var
nan_indices = np.where(np.isnan(metadata_df["qc_flag"]))[0]
metadata_df["fin_flag"] = np.abs(metadata_df["qc_flag"]) > 0
metadata_df.loc[nan_indices, "fin_flag"] = np.nan
fin_presence_df = metadata_df.loc[:, ["genotype", "stage_rounded", "fin_flag"]].groupby(by=["genotype", "stage_rounded"]).agg("mean")
fin_presence_df = fin_presence_df.reset_index()


fig = px.line(fin_presence_df, x="stage_rounded", y="fin_flag", color="genotype", template="plotly")
fig.update_layout(yaxis_title="fraction of embryos with pec fin",
                  xaxis_title="developmental stage (hpf)")
fig.show()
fig.write_image(figure_path + "pecfin_frac_summary.png", scale=2)

### Look at qualitative morphology scores

In [60]:
metadata_df["fin_score"] = np.abs(metadata_df["qc_flag"])
metadata_df.loc[nan_indices, "fin_score"] = np.nan
metadata_df.loc[metadata_df["fin_score"]==0, "fin_score"] = np.nan
fin_score_df = metadata_df.loc[:, ["genotype", "stage_rounded", "fin_score"]].groupby(by=["genotype", "stage_rounded"]).agg("mean")
fin_score_df = fin_score_df.reset_index()

fig = px.line(fin_score_df, x="stage_rounded", y="fin_score", color="genotype", template="plotly")
fig.update_layout(yaxis_title="qualitative morphology score",
                  xaxis_title="developmental stage (hpf)")
fig.update_xaxes(range=[48, 68])
fig.show()
fig.write_image(figure_path + "pecfin_score_summary.png", scale=2)

In [49]:
fin_presence_df.columns = fin_presence_df.columns.to_flat_index()

In [51]:
fin_presence_df.reset_index()

,genotype,stage_rounded,fin_flag
0,prdm1_3_16,24.0,0.0
1,prdm1_3_16,26.0,0.0
2,prdm1_3_16,28.0,0.0
3,prdm1_3_16,30.0,0.0
4,prdm1_3_16,32.0,0.0
...,...,...,...
89,wik-inj-ctrl,62.0,1.0
90,wik-inj-ctrl,64.0,1.0
91,wik-inj-ctrl,66.0,1.0
92,wik-inj-ctrl,68.0,1.0
